# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***


### Import Packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import statistics
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

### Starting Point Functions

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on the image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)

    return masked_image

###  Elaborate Functions

In [3]:
def draw_lines(img, lines, thickness=2):
        
    numLines = len(lines)
    
    if lineType == "solid":
        slopes=[]
        posIndex=[]     # this holds the index position in 'lines' list for a line with a positive slope
        negIndex=[]     # this holds the index position in 'lines' list for a line with a negative slope
        posSlopes=[]    # this holds all the slopes within an appropriate range for left lane lines
        negSlopes=[]    # this holds all the slopes within an appropriate range for right lane lines
            
        for line in range(0,numLines):
            slope= (lines[line][0][3]-lines[line][0][1]) / (lines[line][0][2]-lines[line][0][0])
            slopes.append(slope)
            if ((slope > 0.45) and (slope < 0.85)):
                posIndex.append(line)
                posSlopes.append(slope)
            elif ((slope < -0.45) and (slope > -1.00)):
                negIndex.append(line)
                negSlopes.append(slope)
            else:
                continue
        
        numPos = len(posIndex)
        numNeg = len(negIndex)
        numSlopes = len(slopes)
        
        if numPos != 0:
            medPosSlope = statistics.median(posSlopes)  #this is the number I will use for the right lane line slope
        if numNeg != 0:        
            medNegSlope = statistics.median(negSlopes)  #this is the number I will use for the left lane line slope
    
        #find ideal x and y coordinates for a point on the right lane line
        if numPos != 0:
            posSlopeRef=100 #start with a very high number to make sure it gets replaced
            for i in range(0,numPos):
                if (abs(posSlopes[i]-medPosSlope) < posSlopeRef): #find the line that is closest to the median slope
                    posSlopeRef = abs(posSlopes[i]-medPosSlope)
                    posXRef = (lines[posIndex[i]][0][2] + lines[posIndex[i]][0][0])/2 #save its midpoint
                    posYRef = (lines[posIndex[i]][0][3] + lines[posIndex[i]][0][1])/2
                else:
                    continue
                
        #find ideal x and y coordinates for a point on the left lane line
        if numNeg != 0:
            negSlopeRef=100 #start with a very high number to make sure it gets replaced
            for i in range(0,numNeg):
                if (abs(negSlopes[i]-medNegSlope) < negSlopeRef): #find the line that is closest to the median slope
                    negSlopeRef = abs(negSlopes[i]-medNegSlope)
                    negXRef = (lines[negIndex[i]][0][2] + lines[negIndex[i]][0][0])/2 #save its midpoint
                    negYRef = (lines[negIndex[i]][0][3] + lines[negIndex[i]][0][1])/2
                else:
                    continue
            
        #use one point on the line and the slope to determine two extreme points on the line.
        #if a point lies outside the image boundaries, that's ok
        #because cv2.line() will automatically clip anything outside the image boundaries.
        h,w = (img.shape[:2])   #gives the height and width of the image
    
        #slope = rise/run;  rise = Y2 - Y1;  run = X2 - X1;  slope = (Y2-Y1)/(X2-X1)
        #it is important to note that the lines array's 3rd dimension lists the coordinates [X1,Y1,X2,Y2]
        #where X1 < X2.  This consistency is counted on below.
    
        if numNeg != 0:
            #find the x,y (leftX1,leftY1) coordinate of the near end of the left lane line. leftX1 is unknown.
            lLeftY1 = h #bottom of image
            lLeftX2 = negXRef
            lLeftY2 = negYRef
            #X1 = -(rise/slope)+X2
            lLeftX1 = -(lLeftY2-lLeftY1)/medNegSlope + lLeftX2
            leftX1 = int(lLeftX1) #near end of left lane line x coordinate
            leftY1 = int(lLeftY1) #near end of left lane line y coordinate
    
            #find the x,y (leftX2,leftY2) coordinate of the far end of the left lane line. leftX2 is unknown.
            uLeftX1 = negXRef
            uLeftY1 = negYRef
            uLeftY2 = 0 #top of image
            #X2 = (rise/slope)+X1
            uLeftX2 = (uLeftY2-uLeftY1)/medNegSlope + uLeftX1
            leftX2 = int(uLeftX2) #far end of left lane line x coordinate
            leftY2 = int(uLeftY2) #far end of left lane line y coordinate
    
        if numPos != 0:
            #find the x,y (rightX1,rightY1) coordinate of the far end of the right lane line. rightX1 is unknown
            uRightY1 = 0 #top of image
            uRightX2 = posXRef
            uRightY2 = posYRef
            #X1 = -(rise/slope)+X2
            uRightX1 = -(uRightY2-uRightY1)/medPosSlope + uRightX2
            rightX1 = int(uRightX1) #far end of right lane line x coordinate
            rightY1 = int(uRightY1) #far end of right lane line y coordinate
    
            #find the x,y (rightX2,rightY2) coordinate of the near end of the right lane line. rightX2 is unknown
            lRightX1 = posXRef
            lRightY1 = posYRef
            lRightY2 = h #bottom of image
            #X2 = (rise/slope)+X1
            lRightX2 = (lRightY2-lRightY1)/medPosSlope + lRightX1
            rightX2 = int(lRightX2) #near end of right lane line x coordinate
            rightY2 = int(lRightY2) #near end of right lane line y coordinate
    
        #draw the lines
        if numNeg != 0:
            cv2.line(img, (leftX1,leftY1), (leftX2,leftY2), color, thickness)
        if numPos != 0:
            cv2.line(img, (rightX1,rightY1), (rightX2,rightY2), color, thickness)
    
    else:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)


In [4]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    
    vertices = np.array([[(0,img.shape[0]),(int(img.shape[1]*0.46), int(img.shape[0]*0.60)), (int(img.shape[1]*0.53), int(img.shape[0]*0.60)), (img.shape[1],img.shape[0])]], dtype=np.int32)
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    # check whether HoughLinesP() returned any lines.  If it does not, then lines will have no length
    #len() on a numpy array of no length will throw an error, so if it does we can handle it and move on
    try:
        numLines = len(lines)  
    except TypeError:  
        numLines = 0
    else:
        numLines = len(lines)
    
    #only call draw_lines() if there are lines to draw
    if (numLines>0):
        draw_lines(line_img, lines)
            
    #draw_lines() draws lines across the entire image, so get it back down to the region of interest
    line_img = region_of_interest(line_img, vertices) 
       
    return line_img

In [5]:
def pull_Yellow_White(img):
    #it is easier to extract specific colors in HSV space than BGR or RGB.
    
    hsv_image=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

    yellow_lower = np.array([20, 30, 0])
    yellow_upper = np.array([40, 255, 255])
    yellow = cv2.inRange(hsv_image, yellow_lower, yellow_upper)

    white_lower = np.array([0,0,180])
    white_upper = np.array([255,30,255])
    white = cv2.inRange(hsv_image, white_lower, white_upper)

    yellowWhite = weighted_img(white,yellow)
    
    return yellowWhite

In [6]:
def process_image(image):
        
    if (rgb == 'true'):
        bgrImage = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) 
    else:
        bgrImage = image
    
    #extract yellow and white colors from the image
    yelWhtImage = pull_Yellow_White(bgrImage)
    
    #gaussian blur
    kernel_size=7#5
    blur_yelWht = gaussian_blur(yelWhtImage,kernel_size)

    #canny transformation
    low_threshold = 75#50
    high_threshold = 150#100
    edges = canny(blur_yelWht, low_threshold, high_threshold)

    #region of interest mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(int(imshape[1]*0.46), int(imshape[0]*0.60)), (int(imshape[1]*0.53), int(imshape[0]*0.60)), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    #hough transform
    rho = 2
    theta = np.pi/180
    threshold = 20#30
    min_line_len = 30
    max_line_gap = 20
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    
    #overlay images
    combined = weighted_img(lines, image, α=0.8, β=1., γ=0.) 
    
    ##these lines are helpful when playing around with the parameters
    #plt.imshow(cv2.cvtColor(edges,cv2.COLOR_RGB2BGR))  
    #plt.show()                                           
    
    return combined


## Start With Images


This cell will iterate through the `test_images directory`, find the lane lines in each image, overlay them onto the a copy of the original, and save the renamed results in the `test_images_output` directory.

In [7]:
for im in os.listdir("test_images/"): #iterate through each file name in the subdirectory
    filename = 'test_images/'+ im     #add the subdirectory to the file name
    image = cv2.imread(filename)  #use the path/filename to read the image ##alternate use image = mpimg.imread(filename)
    lineType = "segments" #global variable accessed inside hough_lines()
    color=[0, 0, 255]     #global variable accessed inside draw_lines()
    rgb = 'false'         #global variable accessed inside process_image()
    
    #find the lane lines
    combined = process_image(image)
    
    #save the images
    saveName = 'test_images_output/highlighted_'+im
    cv2.imwrite(saveName,combined)
    
    ##these lines plot images inline for convenience
    #plt.imshow(cv2.cvtColor(combined,cv2.COLOR_RGB2BGR))
    #plt.show()
    #print(filename)

## Test on Videos

This cell performs those same functions, except on a video: `solidWhiteRight.mp4` 
The output is saved in the `test_videos_output` directory.


In [8]:
white_output = 'test_videos_output/highlighted_solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4") #.subclip(2,3) ##add this for partial video
lineType = "segments" 
color=[255, 0, 0]     
rgb = 'true'   
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|▏         | 4/221 [00:00<00:06, 31.20it/s, now=None]

Moviepy - Building video test_videos_output/highlighted_solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/highlighted_solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/highlighted_solidWhiteRight.mp4
CPU times: user 10.4 s, sys: 1.28 s, total: 11.6 s
Wall time: 16.6 s


**This cell can display the video inline:**

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the Lines

**The draw_lines() function has been adapted to identify two solid lines of best fit - one for the left lane line and one for the right.**

This is preformed on the provided video: `solidYellowLeft.mp4` 
The output is saved in the `test_videos_output` directory.


In [9]:
yellow_output = 'test_videos_output/bestFit_solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4') #.subclip(3,5)
lineType = "solid" 
color=[255, 0, 0]     
rgb = 'true'
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|          | 4/681 [00:00<00:21, 31.70it/s, now=None]

Moviepy - Building video test_videos_output/bestFit_solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/bestFit_solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/bestFit_solidYellowLeft.mp4
CPU times: user 31.6 s, sys: 3.69 s, total: 35.3 s
Wall time: 44.6 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

The `challenge.mp4` video presents less than ideal lighting and road colors, which can be overcome with more robust functions.

In [10]:
challenge_output = 'test_videos_output/bestFit_challenge.mp4'
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(4,5) 
clip3 = VideoFileClip('test_videos/challenge.mp4')
lineType = "solid" 
color=[255, 0, 0]     
rgb = 'true'
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   1%|          | 3/251 [00:00<00:10, 23.46it/s, now=None]

Moviepy - Building video test_videos_output/bestFit_challenge.mp4.
Moviepy - Writing video test_videos_output/bestFit_challenge.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/bestFit_challenge.mp4
CPU times: user 19 s, sys: 2.49 s, total: 21.5 s
Wall time: 32.6 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))